In [7]:
#!/usr/bin/env python3
# Sistemas Avançados de Visão Industrial (SAVI 22-23)
# Miguel Riem Oliveira, DEM, UA


from copy import deepcopy
from random import randint
from PIL import Image
import cv2

# --------------------------------------
# Initialization
# --------------------------------------
train_image = cv2.imread("./Images/random_points.png")
query_image = cv2.imread("./Images/random_points.png")

train_image_gui = deepcopy(train_image)
query_image_gui = deepcopy(query_image)

height_t, width_t, nc_t = train_image.shape
height_q, width_q, nc_q = query_image.shape
# print(query_image.shape)
# print(train_image.shape)

# --------------------------------------
# Execution
# --------------------------------------

# Sift features  -----------------------
sift_detector = cv2.SIFT_create(nfeatures=1000)

t_key_points, t_descriptors = sift_detector.detectAndCompute(train_image, None)
q_key_points, q_descriptors = sift_detector.detectAndCompute(query_image, None)

# Match the keypoints
index_params = dict(algorithm = 1, trees = 15)
search_params = dict(checks = 50)
flann_matcher = cv2.FlannBasedMatcher(index_params, search_params)
two_best_matches = flann_matcher.knnMatch(q_descriptors, t_descriptors, k=2)

# Create a list of matches
matches = []
for match_idx, match in enumerate(two_best_matches):

    best_match = match[0] # to get the cv2.DMatch from the tuple [match = (cv2.DMatch)]
    second_match = match[1]

    # David Lowe's ratio
    if best_match.distance < 0.3 * second_match.distance: # this is a robust match, keep it
        matches.append(best_match) # create a list to show with drawMatches

matches_image = cv2.drawMatches(query_image, q_key_points, train_image, t_key_points, matches, None)


# Visualization -----------------------

# Draw the keypoints on the images
for key_point in t_key_points: # iterate all keypoints
    x, y = int(key_point.pt[0]), int(key_point.pt[1])
    color = (randint(0, 255), randint(0, 255),randint(0, 255))
    cv2.circle(train_image_gui, (x,y), 15, color, 1)

for key_point in q_key_points: # iterate all keypoints
    x, y = int(key_point.pt[0]), int(key_point.pt[1])
    color = (randint(0, 255), randint(0, 255),randint(0, 255))
    cv2.circle(query_image_gui, (x,y), 15, color, 1)

train_image_gui = cv2.cvtColor(train_image_gui, cv2.COLOR_BGR2RGB)
imtrain_pil = Image.fromarray(train_image_gui)
query_image_gui = cv2.cvtColor(query_image_gui, cv2.COLOR_BGR2RGB)
imquery_pil = Image.fromarray(query_image_gui)
matches_image = cv2.cvtColor(matches_image, cv2.COLOR_BGR2RGB)
matches_pil = Image.fromarray(matches_image)
# cv2.namedWindow('train image', cv2.WINDOW_NORMAL)
# cv2.imshow('train image', train_image_gui)

# cv2.namedWindow('query image', cv2.WINDOW_NORMAL)
# cv2.imshow('query image', query_image_gui)

# cv2.namedWindow('matches image', cv2.WINDOW_NORMAL)
# cv2.imshow('matches image', matches_image)

# cv2.waitKey(0)
# --------------------------------------
# Termination
# --------------------------------------

In [10]:
matches_pil.save('matches_example.png')

In [11]:
!pip install nbzip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 KB 601.0 kB/s eta 0:00:00a 0:00:01
